# load_channels_static_to_bigquery

### 🎯Objetivo

Este notebook no transforma datos.
Este notebook tiene una única responsabilidad:
Cargar el DataFrame df_channels_snapshot (generado en el notebook 03) hacia BigQuery como tabla stática 

Destino:
youtube-datasets-360.angelgarciadatablog.channels_static

In [ ]:
from dotenv import load_dotenv
import os
from google.cloud import bigquery

load_dotenv()

PROJECT_ID = os.getenv("GCP_PROJECT")
DATASET_ID = "angelgarciadatablog"
TABLE_ID = "channels_static"

FULL_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

client = bigquery.Client(project=PROJECT_ID)

print("Destino configurado:", FULL_TABLE_ID)

Destino configurado: youtube-datasets-360.angelgarciadatablog.channels_static


## 🧱 Cargar snapshot desde Parquet (temporal)

⚠️ Nota temporal:
Durante la fase de notebooks, el DataFrame se carga desde Parquet como mecanismo de intercambio entre notebooks.
En la versión productiva (scripts .py), el DataFrame se pasará directamente sin almacenamiento intermedio.

In [3]:
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path.cwd().parents[0]
PROCESSED_PATH = PROJECT_ROOT / "data" / "processed" / "youtube"

file_path = PROCESSED_PATH / "channels_static.parquet"

df_channels_static = pd.read_parquet(file_path)

df_channels_static.head()


,channel_id,channel_title,description,country,published_at,thumbnail_url,channel_url,extracted_at
0,UCUEOHBht8pnQhQvCfIcl-gg,Angel García,"Hola, me llamo Angel.\nEn este canal encontrar...",PE,2017-02-10 22:27:16+00:00,https://yt3.ggpht.com/wyD9NcUpCMtw3KboShIIPSKx...,https://www.youtube.com/channel/UCUEOHBht8pnQh...,2026-02-14 20:40:58.841008+00:00


In [5]:
df_channels_static.dtypes

channel_id                       str
channel_title                    str
description                      str
country                          str
published_at     datetime64[us, UTC]
thumbnail_url                    str
channel_url                      str
extracted_at     datetime64[us, UTC]
dtype: object

## 🏗 Crear tabla particionada con el esquema y datos del dataframe 

In [ ]:
#Crear la tabla solo la primera vez, luego ya no se creará

from google.api_core.exceptions import NotFound
from google.cloud.bigquery import SchemaField

schema = [
    SchemaField("channel_id", "STRING"),
    SchemaField("channel_title", "STRING"),
    SchemaField("description", "STRING"),
    SchemaField("country", "STRING"),
    SchemaField("published_at", "TIMESTAMP"),
    SchemaField("thumbnail_url", "STRING"),
    SchemaField("channel_url", "STRING"),
    SchemaField("extracted_at", "TIMESTAMP"),
]

try:
    client.get_table(FULL_TABLE_ID)
    print("Tabla ya existe.")
    
except NotFound:
    table = bigquery.Table(FULL_TABLE_ID, schema=schema)
    client.create_table(table)
    print("Tabla creada.")


Tabla creada.


## 📌 Cargar datos del parquet a big query

In [7]:
# 2️⃣ Carga los datos desde tu DataFrame hacia BigQuery. WRITE TRUNCATE = reescribe los datos
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE"
)

job = client.load_table_from_dataframe(
    df_channels_static,
    FULL_TABLE_ID,
    job_config=job_config
)

job.result()

print("Tabla static reemplazada correctamente.")

/Users/angelgarciachanga/repositorios/publico/youtube-v3-data-pipeline/venv/lib/python3.11/site-packages/google/cloud/bigquery/_pandas_helpers.py:484: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


Tabla static reemplazada correctamente.
